In [1]:
import time
from selenium.webdriver.common.keys import Keys
from selenium import webdriver
from bs4 import BeautifulSoup
import re
import pandas as pd
import os
from selenium.webdriver.support.ui import Select

In [2]:
class stock:
    def __init__(self, num, vol, value, avg_price, owner_change):
        self.num = num
        self.vol = vol
        self.value = value
        self.avg_price = avg_price
        self.owner_change = owner_change

In [3]:
def month_maker(month):
    month = month.replace("فروردین", '01')
    month = month.replace("اردیبهشت", '02')
    month = month.replace("خرداد", '03')
    month = month.replace("تیر", '04')
    month = month.replace("مرداد", '05')
    month = month.replace("شهریور", '06')
    month = month.replace("مهر", '07')
    month = month.replace("آبان", '08')
    month = month.replace("آذر", '09')
    month = month.replace("دی", '10')
    month = month.replace("بهمن", '11')
    month = month.replace("اسفند", '12')
    return month

In [4]:
def date_maker(year, month, day):
    date = year + '-' + month + '-' + day
    return date

In [5]:
def value_fixer(val):
    if '.' in val:
        result = float(val.replace(",", ""))
    else:
        result = int(val.replace(",", ""))
    return result

In [6]:
def value_finder(val):
    result = value_fixer(val.find("div").get("title"))
    return result

In [7]:
haghighi_buy = stock([], [], [], [], [])
haghighi_sell = stock([], [], [], [], [])
hoghoghi_buy = stock([], [], [], [], [])
hoghoghi_sell = stock([], [], [], [], [])

In [8]:
code = '9211775239375291'
link = 'http://www.tsetmc.com/Loader.aspx?ParTree=151311&i=' + code

In [9]:
driver = webdriver.Chrome('chromedriver.exe')
driver.maximize_window()
xpath1 = '''//*[@id="tabs"]/div/ul/li[9]/a'''
driver.get(link)
element = driver.find_element_by_xpath(xpath1)
element.click()
time.sleep(3)

In [10]:
page_source = driver.page_source
soup = BeautifulSoup(page_source, 'lxml')

In [11]:
table = soup.find(id = 'ClientTypeBody')
data = table.find_all("td")

In [12]:
dSize = [] #each 25 lines of source is one day of data
for i in range(0, len(data)+1, 25):
    dSize.append(i)

In [13]:
Data = []
for i in range(len(dSize)-1):
    Data.append(data[dSize[i]:dSize[i+1]])

In [14]:
year = soup.find(id='YearPart').text

months = []
for i in range(len(Data)):
    months.append(Data[i][0].find_all("div")[0].text)
    
days = []
for i in range(len(Data)):
    days.append(Data[i][0].find_all("div")[1].text)
    
for i in range(len(months)):
    months[i] = month_maker(months[i])

dates = []
for i in range(len(months)):
    dates.append(date_maker(year, months[i], days[i]))

In [15]:
number = []
for d in Data[0][2:6]:
    number.append(value_fixer(d.text))

In [16]:
volume = []
for d in Data[0][7:11]:
    volume.append(value_finder(d))

In [17]:
value = []
for d in Data[0][12:16]:
    value.append(value_finder(d))

In [18]:
avg_price = []
for d in Data[0][17:21]:
    avg_price.append(value_fixer(d.text))

In [19]:
owner_change = []
owner_change.append(value_finder(Data[0][23]))

# Creating data for whole table

In [20]:
num_temp = []
num = []

vol_temp = []
vol = []

value_temp = []
value = []

avg_price_temp = []
avg_price = []

owner_change = []

for i in range(len(Data)):
    for d in Data[i][2:6]:
        num_temp.append(value_fixer(d.text))
    num.append(num_temp)
    num_temp = []
    
    for d in Data[i][7:11]:
        vol_temp.append(value_finder(d))
    vol.append(vol_temp)
    vol_temp = []
    
    for d in Data[i][12:16]:
        value_temp.append(value_finder(d))
    value.append(value_temp)
    value_temp = []
    
    for d in Data[i][17:21]:
        avg_price_temp.append(value_fixer(d.text))
    avg_price.append(avg_price_temp)
    avg_price_temp = []
    
    owner_change.append(value_finder(Data[i][23]))
    
    
    haghighi_buy.num.append(num[i][0])
    hoghoghi_buy.num.append(num[i][1])
    haghighi_sell.num.append(num[i][2])
    hoghoghi_sell.num.append(num[i][3])
    
    haghighi_buy.vol.append(vol[i][0])
    hoghoghi_buy.vol.append(vol[i][1])
    haghighi_sell.vol.append(vol[i][2])
    hoghoghi_sell.vol.append(vol[i][3])
    
    haghighi_buy.value.append(value[i][0])
    hoghoghi_buy.value.append(value[i][1])
    haghighi_sell.value.append(value[i][2])
    hoghoghi_sell.value.append(value[i][3])
    
    haghighi_buy.avg_price.append(avg_price[i][0])
    hoghoghi_buy.avg_price.append(avg_price[i][1])
    haghighi_sell.avg_price.append(avg_price[i][2])
    hoghoghi_sell.avg_price.append(avg_price[i][3])
    
    haghighi_buy.owner_change.append(owner_change[i])
    hoghoghi_buy.owner_change.append(owner_change[i])
    haghighi_sell.owner_change.append(owner_change[i])
    hoghoghi_sell.owner_change.append(owner_change[i])

# Change iranian date to christian date

In [21]:
import os
os.chdir("C:/Users/Light/Jupyter/Web scraper")
import PersianCal

In [22]:
per_date = []
for date in dates:
    per_date.append(PersianCal.Persian(date).christian_string())

In [26]:
df1 = {"Date": per_date,
      "Number": haghighi_buy.num,
      "Volume": haghighi_buy.vol,
      "Value": haghighi_buy.value,
      "Average Price": haghighi_buy.avg_price,
      "Owner Change": haghighi_buy.owner_change}

In [27]:
dff1 = pd.DataFrame(df1)

In [28]:
dff1.Date = pd.to_datetime(dff1["Date"], format = '%Y-%m-%d')

In [29]:
dff1 = dff1.sort_values('Date')

In [ ]:
# dff1.set_index("Date", inplace = True)

In [ ]:
# dff1.to_csv("pp.csv")

In [30]:
dff1

,Date,Number,Volume,Value,Average Price,Owner Change
95,2020-03-24,8587,193725370,502628674741,2594.54,18892364
94,2020-03-25,7988,195181405,520190153171,2665.16,285589
93,2020-03-28,8614,248921949,685734528376,2754.82,35123935
92,2020-03-29,12113,314731754,896797504482,2849.40,33912528
91,2020-03-30,6493,203954429,582831332761,2857.65,-60228977
...,...,...,...,...,...,...
4,2020-08-12,16784,163750982,1544418794674,9431.51,-5724172
3,2020-08-15,6837,62974940,562426314416,8930.95,-2332464
2,2020-08-16,7152,93906162,796937740851,8486.53,-7480645
1,2020-08-17,22056,271017598,2205390692705,8137.44,-27124329
